# Fedbiomed Researcher to train a model on a CSV dataset

Use for developing (autoreloads changes made across packages)

In [ ]:
%load_ext autoreload
%autoreload 2

This example shows how to use a CSV format file as a node dataset. The example CSV file is synthetic data with a format inspired from ADNI dataset.

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset
    * use `#test_data` for the tags
  * Pick the .csv file from your PC (here: [pseudo_adni_mod.csv](./data/CSV/pseudo_adni_mod.csv))
  * Data must have been added
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node start`. Wait until you get `Starting task manager`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torch.utils.data import Dataset
import pandas as pd

# Here we define the model to be used. 
# You can use any class name (here 'MyTrainingPlan')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        # should match the model arguments dict passed below to the experiment class
        self.in_features = model_args['in_features']
        self.out_features = model_args['out_features']
        self.fc1 = nn.Linear(self.in_features, 5)
        self.fc2 = nn.Linear(5, self.out_features)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch Dataset and DataLoader classes
        # We need pandas to read the local .csv file at the node side
        deps = ["from torch.utils.data import Dataset",
                "import pandas as pd"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    def training_step(self, data, target):
        output = self.forward(data).float()
        criterion = torch.nn.MSELoss()
        loss   = criterion(output, target.unsqueeze(1))
        return loss

    def training_data(self,  batch_size = 48):
        
        df = pd.read_csv(self.dataset_path, sep=';', index_col=False)
        x_dim = self.in_features
        x_train = df.iloc[:,:x_dim].values
        y_train = df.iloc[:,-1].values
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        
        data_manager = DataManager(dataset=x_train , target=y_train, **train_kwargs)
        
        return data_manager

In [2]:
# model parameters 
model_args = {
    'in_features': 15, 
    'out_features': 1
}

# training parameters 
training_args = {
    'batch_size': 20, 
    'lr': 1e-3, 
    'epochs': 10, 
    'dry_run': False,  
    #'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

# Calling the training data with specified tags
tags =  ['#test_data']
rounds = 5

exp = Experiment(tags=tags,
                 model_class=MyTrainingPlan,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-16 13:21:32,066 fedbiomed INFO - Component environment:
2022-03-16 13:21:32,067 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-16 13:21:32,234 fedbiomed INFO - Messaging researcher_96a37edc-2ba8-47d7-aa8e-33679104e4b2 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f02d458df70>
2022-03-16 13:21:32,266 fedbiomed INFO - Searching dataset with data tags: ['#test_data'] for all nodes
2022-03-16 13:21:32,268 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Message received: {'researcher_id': 'researcher_96a37edc-2ba8-47d7-aa8e-33679104e4b2', 'tags': ['#test_data'], 'command': 'search'}
2022-03-16 13:21:42,305 fedbiomed INFO - Node selected for training -> node_19ef0050-617d-4624-bbce-207469edf883
2022-03-16 13:21:42,337 fedbiomed DEBUG - Model file has been saved: /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0090/my_model_c716fed1-2d5b-4681-a2dc-ed7c6b78e

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [4]:
exp.run()

2022-03-16 13:21:55,584 fedbiomed INFO - Sampled nodes in round 0 ['node_19ef0050-617d-4624-bbce-207469edf883']
2022-03-16 13:21:55,589 fedbiomed INFO - Send message to node node_19ef0050-617d-4624-bbce-207469edf883 - {'researcher_id': 'researcher_96a37edc-2ba8-47d7-aa8e-33679104e4b2', 'job_id': 'b928778d-4876-42df-a7c5-3a286357555f', 'training_args': {'batch_size': 20, 'lr': 0.001, 'epochs': 10, 'dry_run': False}, 'model_args': {'in_features': 15, 'out_features': 1}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/16/my_model_c716fed1-2d5b-4681-a2dc-ed7c6b78e763.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/16/aggregated_params_init_f56d01ac-8662-4acb-bb20-64e1cc7f59f4.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_19ef0050-617d-4624-bbce-207469edf883': ['dataset_20497cd4-242e-4320-bf90-87a33f304884']}}
2022-03-16 13:21:55,592 fedbiomed DEBUG - researcher_96a37edc-2ba8-47d7-aa8e-33679104e4b2
2022-03-16 13:21:55,600 fedbiom

2022-03-16 13:21:55,942 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 7 [20] Completed (0%)]	 Metric[Loss] 187.949188
2022-03-16 13:21:55,951 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 7 [20] Completed (25%)]	 Metric[Loss] 223.947235
2022-03-16 13:21:55,961 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 7 [20] Completed (50%)]	 Metric[Loss] 250.577591
2022-03-16 13:21:55,970 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 7 [20] Completed (75%)]	 Metric[Loss] 167.154114
2022-03-16 13:21:55,978 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 8 [20] Completed (0%)]	 Metric[Loss] 178.967392
2022-03-16 13:21:55,990 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 8 [20] Completed (25%)]	 Metric[Loss] 177.848328
2022-03-16 13:21:56,002 fedbiomed IN

2022-03-16 13:22:05,731 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Using device cpu for training (cuda_available=False, gpu=False, gpu_only=False, use_gpu=False, gpu_num=None)
2022-03-16 13:22:05,733 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 1 [20] Completed (0%)]	 Metric[Loss] 93.803329
2022-03-16 13:22:05,748 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 1 [20] Completed (25%)]	 Metric[Loss] 216.683624
2022-03-16 13:22:05,762 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 1 [20] Completed (50%)]	 Metric[Loss] 153.177582
2022-03-16 13:22:05,774 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 1 [20] Completed (75%)]	 Metric[Loss] 244.449097
2022-03-16 13:22:05,785 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 2 [20] Completed (0%)]	 Metric[Los

2022-03-16 13:22:15,728 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_48d2b0ce-27c2-4e75-8bee-c229f25b2823.pt successful, with status code 200
2022-03-16 13:22:15,736 fedbiomed INFO - Nodes that successfully reply in round 1 ['node_19ef0050-617d-4624-bbce-207469edf883']
2022-03-16 13:22:15,798 fedbiomed DEBUG - upload (HTTP POST request) of file /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0090/aggregated_params_3a4d8fb4-ff21-4e90-b13d-c8331bd9e5b4.pt successful, with status code 201
2022-03-16 13:22:15,798 fedbiomed INFO - Saved aggregated params for round 1 in /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0090/aggregated_params_3a4d8fb4-ff21-4e90-b13d-c8331bd9e5b4.pt
2022-03-16 13:22:15,799 fedbiomed INFO - Sampled nodes in round 2 ['node_19ef0050-617d-4624-bbce-207469edf883']
2022-03-16 13:22:15,800 fedbiomed INFO - Send message to node node_19ef0050-617d-4624-bbce-207469edf883 - {'researcher_id': 'rese

2022-03-16 13:22:16,023 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 5 [20] Completed (75%)]	 Metric[Loss] 74.918228
2022-03-16 13:22:16,031 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 6 [20] Completed (0%)]	 Metric[Loss] 80.715141
2022-03-16 13:22:16,038 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 6 [20] Completed (25%)]	 Metric[Loss] 40.568268
2022-03-16 13:22:16,046 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 6 [20] Completed (50%)]	 Metric[Loss] 82.535385
2022-03-16 13:22:16,053 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 6 [20] Completed (75%)]	 Metric[Loss] 92.883804
2022-03-16 13:22:16,060 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 7 [20] Completed (0%)]	 Metric[Loss] 86.621849
2022-03-16 13:22:16,067 fedbiomed INFO - T

...

2022-03-16 16:29:27,135 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Train Epoch: 1 [2832/48000 (6%)]	Loss: 0.564436
2022-03-16 16:29:27,137 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 1 [48] Completed (6%)]	 Metric[Loss] 0.564436
2022-03-16 16:29:27,890 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Train Epoch: 1 [3312/48000 (7%)]	Loss: 0.482435
2022-03-16 16:29:27,892 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 1 [48] Completed (7%)]	 Metric[Loss] 0.482435
2022-03-16 16:29:28,595 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Train Epoch: 1 [3792/48000 (8%)]	Loss: 0.326218
2022-03-16 16:29:28,596 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 1 [48] Completed (8%)]	 Metric[Loss] 0.326218
2022-03-16 16:29:29,118 fedbiomed INFO - log from: node_19ef0050-617d-462

2022-03-16 16:30:45,590 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Train Epoch: 1 [4272/48000 (9%)]	Loss: 0.218509
2022-03-16 16:30:45,592 fedbiomed INFO - TRAINING on NODE_ID:[node_19ef0050-617d-4624-bbce-207469edf883] Epoch: 1 [48] Completed (9%)]	 Metric[Loss] 0.218509
2022-03-16 16:30:46,130 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Reached 100 batches for this epoch, ignore remaining data
2022-03-16 16:30:46,139 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - running model.postprocess() method
2022-03-16 16:30:46,140 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - model.postprocess() method not provided
2022-03-16 16:30:46,307 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - upload (HTTP POST request) of file /home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/node_params_ceb0d0de-f354-4bf2-8661-f82bf7b88308.pt s

2022-03-16 16:33:48,337 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - model.postprocess() method not provided
2022-03-16 16:33:48,511 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - upload (HTTP POST request) of file /home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/node_params_7d04ceca-227f-492e-b794-1560edfd246c.pt successful, with status code 201
2022-03-16 16:33:48,511 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / INFO - results uploaded successfully 
2022-03-16 16:34:12,935 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / INFO - Starting task manager
2022-03-16 16:34:24,146 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Message received: {'researcher_id': 'researcher_96a37edc-2ba8-47d7-aa8e-33679104e4b2', 'job_id': '656024e9-9d62-420a-bfa9-c30a3661fc56', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False,

2022-03-16 16:36:46,259 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_96a37edc-2ba8-47d7-aa8e-33679104e4b2', 'job_id': '656024e9-9d62-420a-bfa9-c30a3661fc56', 'params_url': 'http://localhost:8844/media/uploads/2022/03/16/aggregated_params_081d2395-5b68-4a99-9899-7489d79c6e5e.pt', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training_data': {'node_19ef0050-617d-4624-bbce-207469edf883': ['dataset_ba55374f-ddc3-4f5d-8bb6-deac79c459ee']}, 'model_args': {}, 'model_url': 'http://localhost:8844/media/uploads/2022/03/16/my_model_e8f25519-9982-4da7-a176-3ced840dd3d5.py', 'model_class': 'MyTrainingPlan', 'command': 'train'}
2022-03-16 16:36:46,269 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - upload (HTTP GET request) of file my_model_523be68f1fdf48ef8364d1c3d8763f98.py successful, with status code 200
2022-03-16 16:36:46,292

2022-03-16 16:39:52,683 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - upload (HTTP GET request) of file my_model_6d138625-d71f-4c4d-9ce3-468b41ed697b.pt successful, with status code 200
2022-03-16 16:39:53,026 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / DEBUG - Dataset path has been set as../data
2022-03-16 16:39:53,065 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / INFO - training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7fa68759c550>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}
2022-03-16 16:39:58,529 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / INFO - Using weighted calculation for multiclass F1 SCORE
2022-03-16 16:59:36,446 fedbiomed INFO - log from: node_19ef0050-617d-4624-bbce-207469edf883 / CRITICAL - Node stopped in

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available in `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())
